# This is the cats vs dogs covnet notebook

In [30]:
import cv2
import numpy as np
import os
from random import shuffle
from tqdm import tqdm

TRAIN_DIR = '/home/vik/Desktop/DEEP_LEARNING_Projects/CATS_DOGS_DATA/train'
TEST_DIR = '/home/vik/Desktop/DEEP_LEARNING_Projects/CATS_DOGS_DATA/test'
IMG_SIZE = 50   ## this is what we resize the image too 50 x 50
LR = 1e-3

MODEL_NAME = 'dogsvscats--{}--{}.model'.format(LR,'6ConvLayer')    ## USEFUL FOR SAVING THE MODEL

In [31]:
# HELPS CREATE ON HOT LABELS FOR THE CAT AND DOGS
def label_img(img):
    word_label = img.split('.')[-3]
    if word_label == 'cat':return [1,0]
    elif word_label == 'dog':return [0,1]

In [32]:
# CREATE A TRAINING DATA WITH GRAYSCALED AND RESIZED IMAGE DATA AND RESPECTIVE ONE HOT LABELS.
def create_train_data():
    training_data = []
    for img in tqdm(os.listdir(TRAIN_DIR)):
        label = label_img(img)
        path = os.path.join(TRAIN_DIR,img)
        img = cv2.resize(cv2.imread(path, cv2.IMREAD_GRAYSCALE), (IMG_SIZE, IMG_SIZE))
        training_data.append([np.array(img), np.array(label)])
    shuffle(training_data) # training data shuffled to bring in the randomness
    np.save('train_data.npy', training_data) # saving the training data so that we don't have to run this loop again
    return training_data

In [33]:
#train_data = create_train_data()
train_data=np.load('train_data.npy') # testing to see what the data looks like

In [34]:
print train_data[0]

[ array([[237, 212, 128, ..., 201, 201, 197],
       [239, 212, 143, ..., 202, 199, 197],
       [242, 216, 141, ..., 203, 199, 198],
       ..., 
       [179, 198, 180, ..., 177, 165, 164],
       [197, 197, 232, ..., 138, 204, 232],
       [218, 232, 237, ..., 151, 106, 173]], dtype=uint8)
 array([1, 0])]


In [35]:
def process_test_data():
    testing_data = []
    for img in tqdm(os.listdir(TEST_DIR)):
        img_num = img.split('.')[0]
        path = os.path.join(TEST_DIR,img)
        img = cv2.resize(cv2.imread(path, cv2.IMREAD_GRAYSCALE), (IMG_SIZE, IMG_SIZE))
        testing_data.append([np.array(img), img_num])
    np.save('test_data.npy', testing_data) # saving the testing data so that we don't have to run this loop again
    return testing_data   

In [19]:
#process_test_data()

In [36]:
import tflearn
from tflearn.layers.conv import conv_2d, max_pool_2d
from tflearn.layers.core import input_data, dropout, fully_connected
from tflearn.layers.estimator import regression

import tensorflow as tf
tf.reset_default_graph()

convnet = input_data(shape=[None, IMG_SIZE, IMG_SIZE, 1], name='input')

convnet = conv_2d(convnet, 32, 5, activation='relu')
convnet = max_pool_2d(convnet, 5)

convnet = conv_2d(convnet, 64, 5, activation='relu')
convnet = max_pool_2d(convnet, 5)

convnet = conv_2d(convnet, 32, 5, activation='relu')
convnet = max_pool_2d(convnet, 5)

convnet = conv_2d(convnet, 64, 5, activation='relu')
convnet = max_pool_2d(convnet, 5)

convnet = conv_2d(convnet, 32, 5, activation='relu')
convnet = max_pool_2d(convnet, 5)

convnet = conv_2d(convnet, 64, 5, activation='relu')
convnet = max_pool_2d(convnet, 5)

convnet = fully_connected(convnet, 1024, activation='relu')
convnet = dropout(convnet, 0.8)

convnet = fully_connected(convnet, 2, activation='softmax')
convnet = regression(convnet, optimizer='adam', learning_rate=LR, loss='categorical_crossentropy', name='targets')

model = tflearn.DNN(convnet, tensorboard_dir='log')


In [42]:
if os.path.exists('{}.meta'.format(MODEL_NAME)):
    model.load(MODEL_NAME)
    print('model loaded!')

INFO:tensorflow:Restoring parameters from /home/vik/Desktop/DEEP_LEARNING_Projects/dogsvscats--0.001--6ConvLayer.model
model loaded!


In [38]:
train = train_data[:-500]
test = train_data[-500:] # for validation

In [39]:
X = np.array([i[0] for i in train]).reshape(-1, IMG_SIZE, IMG_SIZE, 1) # cause 4d tensor filters will be used, imgs
Y = [i[1] for i in train] # lables

test_x = np.array([i[0] for i in test]).reshape(-1,IMG_SIZE,IMG_SIZE,1)
test_y = [i[1] for i in test]

In [43]:
model.fit({'input': X}, {'targets': Y}, n_epoch=5, validation_set=({'input': test_x}, {'targets': test_y}), 
    snapshot_step=500, show_metric=True, run_id=MODEL_NAME) # run_id is the id of the tensor board run

Training Step: 3829  | total loss: 0.32424 | time: 122.656s
| Adam | epoch: 010 | loss: 0.32424 - acc: 0.8567 -- iter: 24448/24500
Training Step: 3830  | total loss: 0.31701 | time: 123.936s
| Adam | epoch: 010 | loss: 0.31701 - acc: 0.8601 | val_loss: 0.46564 - val_acc: 0.7900 -- iter: 24500/24500
--


In [29]:
# visualizing tensorboard
# run it in the terminal
#tensorboard --logdir=foo:/home/vik/Desktop/DEEP_LEARNING_Projects/log 

In [44]:
model.save(MODEL_NAME)

INFO:tensorflow:/home/vik/Desktop/DEEP_LEARNING_Projects/dogsvscats--0.001--6ConvLayer.model is not in all_model_checkpoint_paths. Manually adding it.
